## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
import mlflow

This will assume that we want to save our files and artifacts on the current folder.

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns'


This will create the mlruns folder. 

In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1685106797118, experiment_id='0', last_update_time=1685106797118, lifecycle_stage='active', name='Default', tags={}>]

Inside the created folder, you can see two sub-folders: .trash and 0, which is the experiment_id. This latter contains information about the experiment saved in an YAML file meta.yaml. 

### Creating an experiment and logging a new run
This code will create a new experiment, so inside the mlruns folders a new folder named after the experiment ID will be created*. Inside it, there will be a new folder for anything you asked mlflow to log, in this case: params, metrics, models (which are save inside the artifacts folder). The tags are also saved. Everything will be save on individual files inside these folders.

*In the video, this folder has ID 1, following the order, but here my created folder has what seems to be a random ID. I don't know why this happened, probably because of some different configuration - remember the video was recorded with an older version of MLflow. 

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/05/26 10:20:45 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'file:///home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/310781406204023939/9eb2a0e59f7b45dfb613fff70f61f796/artifacts'


Now, if we run this command again, we are going to see a new experiment being shown in the output:

In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/310781406204023939', creation_time=1685107245818, experiment_id='310781406204023939', last_update_time=1685107245818, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1685106797118, experiment_id='0', last_update_time=1685106797118, lifecycle_stage='active', name='Default', tags={}>]

Now, if we want to run the MLflow UI, we have to, on the terminal, move to the directory where the mlruns folder of interested is located (in this case, it is in the same folder of this notebook). If you run it on the wrong directory, it will create a new mlruns folder there, or use one that already exists there. 

### Interacting with the model registry

In [13]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [14]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

Finally, we are going to delete the mlruns folder that was created for this scenario in order to explore scenario 2.